# Introduction

![](https://i.pinimg.com/originals/db/4f/88/db4f88f155d22599f59765e14f4c5497.jpg)

## Agenda of this talk

**Below is a typical simplified Machine learning model development pipeline for tabular data**

![](img/fig1.png)

In a typical model development pipeline there is raw data that exists (across servers/schemas etc) which is aggregated to get the exhaustive model development data or data which might be useful to solve the problem at hand .Post this the model development data is used to develop an outcome or the target variable(example:Sales,default,fraud etc) and independent variables which might be useful in predicting the target .The supervised machine learning algorithm uses the independent predictors and the target to develop a predictive entity which helps in getting an estimation for the predictive problem.

**Today's talk is based on how the raw variables(specifically categorical variables) should be transformed for usage into model development for better predictive accuracy and long term maintainance** 

<a id='data_types'></a>
# A quick segway into model development data types 

https://stats.idre.ucla.edu/other/mult-pkg/whatstat/what-is-the-difference-between-categorical-ordinal-and-numerical-variables/

**Model development data** : Data captured for most of the problem statments that you might be trying to solve should fall in one of the below buckets:

- **Categorical variables:**A variable which does not represent a numeric entity or an entity that cannot be represented on a coordinate scale.They need to be transformed into a numeric format for usage in mathematical algorithms
 - **Ordinal variables:**variable  with inherent ranking/ordering
   - Examples:Academic grades(A++,A,A-,..),Age Bracket(New born,Baby,Toddler..)
 - **High cardinality:**variable with unique values which are greater than 15(**My own thumb rule**)
   - Examples: zipcodes,product IDs,Operating system version numbers,Email_domain_address
 - **Low cardinality:**variable with unique values which are less than 15(**My own thumb rule**)
   - Examples: credit_default_status(YES/NO),customer_status(Active/inactive/attrited)
 - **Variables that you might mistake to be numeric variables:**A variable whose values are numbers but does not have an inherent ordering to them
   - Examples : zipcodes,House-numbers,OS version numbers
- **Numeric variables:** A variable which can be represented as a numeric entity or on a coordinate scale.
 - The values that a numeric variable might take might vary depending upon the variable type and can be contiguous,integers,binary.They can be used directly as predictors in mathemarical algorithms
    - Examples :Distance,speed,Income,credit score,Indicator_for_having_a_pet(1/0)
- **Alternate data types**
 - **Text**
 - **Images**
 - **Videos**
 - **Every other damn thing under the blue sky** 🙄

Lets pick up an extremely popular dataset from kaggle to get a feel of the variable types we just encountered.

In [11]:
# Titance dataset:Predict survival on the Titanic
#(An extremly popular and a kind of Hello world dataset within competitive predictive modelling landscape)

import pandas as pd
df=pd.read_csv("data/train.csv")

In [22]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In the dataset we have 12 variables, within which Survived(Who survives the Titanic) is the binary outcome to be predicted.Let's classify each of the other features into one of the above variable classification.


One of the quick ways to identify  a variables type other than business/domain knowledge is to check the data types of variable

In [17]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

Its also worth checking the number of unique values for each variable

In [20]:
df.shape

(891, 12)

In [19]:
df.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

We observe that there are four variables which are classified as object dtype  which is a hint that they are categorical variables(Name,Sex,Cabin,Embarked).
Further,we have two variables which have same number of unique values as number of passengers,indicating they are the primary keys.
Based on that we can classify the 12 variables as:

1.  **PassengerId :** The primary key passenger ID is a High cardinality categorical variable.Although the variable is numeric in form we are not classifying it as numeric as it cannot be used on numeric scale that is passenger ID 1 passenger ID 2 has no meaning.

2. **Survived :** The outcome variable as this is a classification problem is a binary numeric variable(I am classifying it as numeric as it in already encoded as 1/ 0 if it was survived/Not_survived it would have been a low cardinality categorical variable which we would have needed to transform into numeric binary form for development of a classification algorithm

3. **Pclass :** A Low cardinality categorical variable

4. **Name :** A High cardinality categorical variable

5. **Sex :** A Low cardinality categorical variable

6. **Age :** A Numeric variable

7. **SibSp :** # of siblings / spouses aboard the Titanic, A Numeric variable

8. **Parch:** # of parents / children aboard the Titanic,A Numeric variable

9. **Ticket:** Ticket number,A High cardinality categorical variable

10. **Fare:** Passenger fare,A Numeric Variable

11. **Cabin:** Cabin number,A High cardinality categorical variable

12. **Embarked:** Port of Embarkation,A Low cardinality categorical variable



# Transforming categorical variables

## Some guidelines around choosing a categorical variables tranformation methodology

As we mentioned when we were lookinng at typical data types we would face during a predictive development task that categorical variables in their raw form are not usable in a mathematical predictive algorithm and they need to be transformed into a numeric form.

There are various methodologies to conduct the above tranformation for the categorical variables but before we look at them lets define few guidelines around what our final product should be and how we might want to evaluate the results of transformation from categorical to numeric.Below are three major questions that we would ask to evaluate any categorical variable transformation methodology we might find.

- **How much incremental improvement we observe in models predictive strength?**
- **Will the categorical variable transformation methodology be  supported by the technical infrastructure in place for inference of the model in production?**
- **How robust is the methodology against domain shift that we might observe in the data,which would eventually happen in this ever fluctuating world?**

## Methodologies for categorical variable transformation

Below is a list of major variable transformation methodologies used :

- **One Hot encoding**
- **Count encoding**
- **Vanilla target encoding**
- **K-fold Cross validated target encoding**
- **Catboost encoding**

Let's delve into each of them using the Titanic dataset that we encountered in [Section-2](#data_types)


### One Hot encoding

In one hot encoding we transform the categories within the variable into their own individual binary representation.Below example will make it clear.

<u>Below is the Titanic dataset</u>

In [89]:
print("The size of the dataset is {} with {} columns".format(df.shape[0],df.shape[1]))

The size of the dataset is 891 with 12 columns


In [92]:
df['Survived'].value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

<b> The Survival rate is 38% as per the training data<b>

In [23]:

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Let's transform the Sex of the passengers using one Hot encoding,where we will have seperate binary representation for each gender type

In [30]:
df['Sex'].unique()

array(['male', 'female'], dtype=object)

In [29]:
df.join(pd.get_dummies(df['Sex'],prefix='Sex'))[['Sex','Sex_female','Sex_male']].head()

,Sex,Sex_female,Sex_male
0,male,0,1
1,female,1,0
2,female,1,0
3,female,1,0
4,male,0,1


We actually need only n-1 categories to be binarized that is Sex_male in itself captures the information if the sex is female or not.Hence:

In [32]:
df.join(pd.get_dummies(df['Sex'],prefix='Sex',drop_first=True))[['Sex','Sex_male']].head()

,Sex,Sex_male
0,male,1
1,female,0
2,female,0
3,female,0
4,male,1


We would now convert all predictive categorical variables in the dataset into One-hot encoded form and would attempt to develop a quick ML algorithm to predict the survival

First a List of all Categorical variables in the Titanice dataset which heuristically could be predictors of survival of a passenger

In [68]:
Cat_predictors=list(df.drop(['PassengerId','Survived','Ticket',"Name","Age","Fare","SibSp","Parch"],axis=1).columns)


In [69]:
Cat_predictors

['Pclass', 'Sex', 'Cabin', 'Embarked']

<b>The Below one liner in pandas will one hot encode all variables in Cat_predictors<b>

In [104]:
df_OHE=pd.get_dummies(df,columns=Cat_predictors,dummy_na=True,drop_first=True)

In [105]:

df_OHE.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)

<b> Finally,below are the columns in our mock modelling dataset to predict survival on Titanic

In [81]:
list(df_OHE.columns)

['Survived',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Pclass_2.0',
 'Pclass_3.0',
 'Pclass_nan',
 'Sex_male',
 'Sex_nan',
 'Cabin_A14',
 'Cabin_A16',
 'Cabin_A19',
 'Cabin_A20',
 'Cabin_A23',
 'Cabin_A24',
 'Cabin_A26',
 'Cabin_A31',
 'Cabin_A32',
 'Cabin_A34',
 'Cabin_A36',
 'Cabin_A5',
 'Cabin_A6',
 'Cabin_A7',
 'Cabin_B101',
 'Cabin_B102',
 'Cabin_B18',
 'Cabin_B19',
 'Cabin_B20',
 'Cabin_B22',
 'Cabin_B28',
 'Cabin_B3',
 'Cabin_B30',
 'Cabin_B35',
 'Cabin_B37',
 'Cabin_B38',
 'Cabin_B39',
 'Cabin_B4',
 'Cabin_B41',
 'Cabin_B42',
 'Cabin_B49',
 'Cabin_B5',
 'Cabin_B50',
 'Cabin_B51 B53 B55',
 'Cabin_B57 B59 B63 B66',
 'Cabin_B58 B60',
 'Cabin_B69',
 'Cabin_B71',
 'Cabin_B73',
 'Cabin_B77',
 'Cabin_B78',
 'Cabin_B79',
 'Cabin_B80',
 'Cabin_B82 B84',
 'Cabin_B86',
 'Cabin_B94',
 'Cabin_B96 B98',
 'Cabin_C101',
 'Cabin_C103',
 'Cabin_C104',
 'Cabin_C106',
 'Cabin_C110',
 'Cabin_C111',
 'Cabin_C118',
 'Cabin_C123',
 'Cabin_C124',
 'Cabin_C125',
 'Cabin_C126',
 'Cabin_C128',
 'Cabin_C148',

In [106]:
df_OHE.isna().sum().sort_values(ascending=False)

Age              177
Embarked_nan       0
Cabin_B78          0
Cabin_C101         0
Cabin_B96 B98      0
                ... 
Cabin_D20          0
Cabin_D19          0
Cabin_D17          0
Cabin_D15          0
Survived           0
Length: 160, dtype: int64

There are missing values to be imputed in Age.We quickly do a based imputation using median age

In [107]:
df_OHE['Age'].fillna(df['Age'].median(), inplace = True)

In [108]:
from sklearn.model_selection import train_test_split


In [109]:
df_train,df_test=train_test_split(df_OHE,test_size=0.2)

In [110]:
print(df_train.shape,df_test.shape)

(712, 160) (179, 160)


In [95]:
df_train['Survived'].value_counts(normalize=True)

0    0.617978
1    0.382022
Name: Survived, dtype: float64

In [111]:
df_test['Survived'].value_counts(normalize=True)

0    0.620112
1    0.379888
Name: Survived, dtype: float64

Time to develop a quick model and check the predictive quality .For Absolute simplicity for this classification problem we will use a Logistic regression model .There would be probably lots of sighs and roll of eyes 🙄 but come on folks this is a toy problem,we are not trying to beat SOTA 😉

<b> <font color='red'> Note:There is lots of hand waving in the model development steps ignoring steps like correlations,robust missing value imputation and many other fine factors which might influence the scientific quality of a predictive model.We are doing that to be able to capture the flavor of categorical encoding within the stipulated time period.</font>
    

In [121]:
from sklearn.linear_model import LogisticRegression
predictors=df_train.loc[:,df_train.columns!='Survived'].to_numpy()
outcome=df_train['Survived'].to_numpy()
clf = LogisticRegression()

In [122]:
clf.fit(predictors,outcome)

/Users/shubyog/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [117]:
df_train.loc[:,df_train.columns!='Survived'].to_numpy()

array([[28.,  0.,  0., ...,  1.,  0.,  0.],
       [28.,  0.,  0., ...,  1.,  0.,  0.],
       [31.,  1.,  1., ...,  0.,  1.,  0.],
       ...,
       [16.,  0.,  0., ...,  1.,  0.,  0.],
       [60.,  0.,  0., ...,  0.,  1.,  0.],
       [18.,  1.,  0., ...,  0.,  1.,  0.]])